# DT Assignment1

# Data Loading

In [707]:
import pandas as pd 
import numpy as np

In [709]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [710]:
def get_gini(df, label):
    D=df[str(label)].unique() # class 생성
    P=[]
    Gini = 1 # 기본값 1 
    for i in D:
        P.append((len(df[df[str(label)]==i]) / len(df)) **2) # Gini(Di)
    for i in P:
        Gini = Gini - i
    return Gini    

In [711]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [712]:
from itertools import combinations # 조합

In [713]:
def get_binary_split(df, attribute):
    result=[] # 결과값 담는 리스트
    element=df[str(attribute)].unique() # 
    
    # class가 2개 이하면, 그냥 넣어줌    
    if len(element) <= 2:
        for i in element :
            result.append(i)
        
    # 3개 이상일 경우 하나씩 넣어주고, 나머지를 묶어서 넣어줌
    else :
        for i in element :
            result.append(i)
        for i in list(combinations(df[str(attribute)].unique(), (len(element)-1) )):
            result.append(list(i))
   
    
    return result

In [714]:
get_binary_split(pd_data,"age")

['youth',
 'middle_aged',
 'senior',
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [715]:
def get_attribute_gini_index(df, attribute, label):
    D_list=get_binary_split(df,str(attribute)) #Di 생성
    result_dict={} # 결과값 담는 딕셔너리
    for D in D_list:
        df1=df[df[str(attribute)].apply(lambda x: x in D)] # 분할 과정
        df2=df[df[str(attribute)].apply(lambda x: x not in D)] # 분할 과정
        w1=len(df1)/len(df1+df2)# 가중치 생성
        w2=len(df2)/len(df1+df2)# 가중치 생성
        D1=get_gini(df1,str(label)) # 상황별 지니계수 계산
        D2=get_gini(df2,str(label)) # 상황별지니계수 계산
        result=w1*D1+w2*D2 # class의 지니계수 계산
        if isinstance(D,list):
            D='_'.join(D) # key값 만들기
        result_dict[D]=result # 저장
    return(result_dict)   

In [716]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [717]:
##문제1 답안
get_binary_split(pd_data,"income")

['high',
 'medium',
 'low',
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [718]:
##문제2 답안
Gini={}
for col in pd_data.columns[:4]:
    Gini[col]=(min(get_attribute_gini_index(pd_data, str(col), "class_buys_computer").items()))
display(Gini) 
# Age변수의 gini계수가 가장 낮다 -> Age변수가 split할 때 가장 유의미한 변수

{'age': ('middle_aged', 0.35714285714285715),
 'income': ('high', 0.4428571428571429),
 'student': ('no', 0.3673469387755103),
 'credit_rating': ('excellent', 0.42857142857142855)}

In [719]:
##문제3 답안
def DT_Split(df, attribute, class_name):
    df1=df[df[str(attribute)].apply(lambda x: x in class_name)]
    df2=df[df[str(attribute)].apply(lambda x: x not in class_name)]
    return df1,df2

In [720]:
df1,df2 = DT_Split(pd_data,'age','middle_aged')

In [721]:
Gini={}
for col in pd_data.columns[:4]:
    Gini[col]=(min(get_attribute_gini_index(df1, str(col), "class_buys_computer").items()))
display(Gini)
#이미 target변수가 완벽하게 나뉘어져서, Gini계수가 0으로 계산된다.

{'age': ('middle_aged', 0.0),
 'income': ('high', 0.0),
 'student': ('no', 0.0),
 'credit_rating': ('excellent', 0.0)}

In [722]:
Gini={}
for col in pd_data.columns[:4]:
    Gini[col]=(min(get_attribute_gini_index(df2, str(col), "class_buys_computer").items()))
display(Gini)
#student의 gini계수가 가장 낮다 -> student가 split할때 가장 유의미한 변수

{'age': ('senior', 0.48),
 'income': ('high', 0.375),
 'student': ('no', 0.31999999999999984),
 'credit_rating': ('excellent', 0.4166666666666667)}

# Q2. ID3 풀어보기

In [723]:
df=pd_data.copy() # 편의를 위해 복사 

In [724]:
origin_label=df['class_buys_computer'].value_counts() # 원 데이터셋 엔트로피 계산과성

In [725]:
ent_label=0
for c in origin_label :
    p = (c/origin_label.sum())
    ent_label= ent_label + -(p)*np.log2(p)    

In [726]:
ent_label # 원 데이터셋 엔트로피

0.9402859586706311

In [727]:
#함수 생성
def compute_ent(df,target) :
    ent=0
    prob_table=df[str(target)].value_counts()
    for c in prob_table:
        p = (c/prob_table.sum())
        ent = ent + -(p) * np.log2(p)
    return ent

In [728]:
compute_ent(df,'class_buys_computer')

0.9402859586706311

## Gain_a = ent_label - ent_a

## Student 계산

In [729]:
df['student'].unique() # class 수

w1=len(df[df['student']=='yes'])/len(df)
w2=1-w1  #class가 2개이므로
display(w1,w2)

#엔트로피 계산
info_stu_y=compute_ent(df[df['student']=='yes'],'class_buys_computer')
info_stu_n=compute_ent(df[df['student']=='no'],'class_buys_computer')
display(info_stu_y,info_stu_n)

#정보이득 계산
ent_label - (w1*info_stu_y + w2 * info_stu_n)

0.5

0.5

0.5916727785823275

0.9852281360342515

0.15183550136234159

## 일반화 함수

In [730]:
def compute_infogain(df,label):
    ent_label = compute_ent(df,str(label))
    cols=df.columns
    cols=cols[cols!=label]
    for col in cols:
        print(col)
        D_list=df[str(col)].unique()
        df_list=[]
        w_list=[]
        ent_list=[]
        info_gain=[]
        
        for D in D_list:
            df_list.append( df[df[str(col)]==D] )
        for sub in df_list:
            w_list.append(len(sub)/len(df))
            ent_list.append(compute_ent(sub,str(label)))
            #print(w_list)
            #print(ent_list)
        for i in range(len(df_list)):
            info_gain.append(w_list[i] * ent_list[i])
        print (ent_label - sum(info_gain))

In [731]:
compute_infogain(df,'class_buys_computer')
#age가 가장 정보이득이 크다 

age
0.24674981977443933
income
0.02922256565895487
student
0.15183550136234159
credit_rating
0.04812703040826949
